In [7]:
# Importare le librerie
import tweepy
from dotenv import dotenv_values
# Importare le variabili d'ambiente dal file .env
config = dotenv_values(".env")

In [8]:
# %load utils.py
import json
import os


def serealize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder,exist_ok=True)
    with open(f"{folder}/{filename}.json", "w", encoding="utf-8") as f:
        json.dump(data,f,ensure_ascii=False,indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}.json")

def read_json(path):
    try:
        if os.path.exists(path):
            with open(path, "r", encoding="utf-8") as file:
                data = json.load(file)
            return data
    except ValueError:
        print("Path not found, check the correctness of the path")

## Punto 1

Creiamo un client Tweepy utilizzando il `BREARER_TOKEN` presente nel file `.env` e caricato tramite la libreria `dotenv`.

In [9]:
# Nuovo client Tweepy
client = tweepy.Client(bearer_token=config["BEARER_TOKEN"])

Utilizzando la funzione `get_user()` inviamo la richiesta per ricavare le informazioni di default, aggiungiamo inoltre le opportune `user_fields` per ricavare ulteriori informazioni dell'utente, come descrizione, stato di protezione, metriche pubbliche.  
Ricaviamo l'utente `kevin` dal corpo della risposta.

In [26]:
# Fetching dell'utente @KevinRoitero
kevin = client.get_user(username="KevinRoitero").data
# Ricavo i followers di Kevin
followers = client.get_users_followers(id=kevin.id).data
# Inizializzo una lista dove andrò a mettere i follower con i dati aggiuntivi
follower_list = []

for follower in followers:
    user = client.get_user(id=follower.id, user_fields=["public_metrics", "protected", "description"]).data
    user_dict = {
        'id': user.id,
        'name': user.name,
        'username': user.username,
        'description': user.description,
        'public_metrics': user.public_metrics,
        'protected': user.protected
    }
    follower_list.append(user_dict)

# Serealizzo la lista in un file json
serealize_json("out", "Kevin", follower_list)

Data serialized to path: out/Kevin.json


## Punto 2

Tramite la funzione `get_users_followers()` ricavo la lista dei follower che seguono l'utente `kevin`. Successivamente per ogni follower, tramite `get_recent_tweets_count()`, ricavo, con scadenza giornaliera, il numero dei tweets recenti (negli ultimi 7 giorni). Per ricavare un valore unico settimanale, sommo i valori giornalieri in una variabile denominata `last_week_tweets_count`. Infine creo una directory temporanea, rappresentativa del follower, aggiundo varie informazioni riguardante esso, come: id, nome, username e il numero di tweets recenti (calcolato precedentemente). Aggiungo la directory ad una lista precedentemente inizializzata e successivamente aggiunto tale lista all'interno della directory principale `kevin_dict` all'interno della voce `followers`.

In [27]:
# Per ogni follower del profilo ricavare il numero di tweet pubblicati da quel profilo durante l'ultima settimana
for follower in follower_list:
    last_week_tweets_count = 0
    recent_tweets_count = client.get_recent_tweets_count(query=f"from:{follower['username']} -is:retweet",granularity="day").data
    for day in recent_tweets_count or []:
            last_week_tweets_count += day["tweet_count"]
    follower['last_week_tweets_count'] = last_week_tweets_count #aggiungo alle info di follower anche last_week_tweets_count
# Aggiungere il numero di tweets nel JSON
serealize_json("out","Kevin", follower_list)

Data serialized to path: out/Kevin.json


## Punto 3

In [29]:
follower_filtrati = []  #filtro la lista di follower di kevin scegliendo solo i follower che hanno a lro volta almeno un follower e non sono protected

for follower in follower_list: 
    
    follower_count = follower["public_metrics"]["followers_count"]
    protected = follower["protected"]
   
    if (follower_count > 1 and protected == False):
        follower_filtrati.append(follower)



all_followers = {}  
for current_user in follower_filtrati: #per ogni utente tra gli utenti della lista filtrata di followers di kevin
    response = client.get_user(username=current_user) #lo identifico
    user_id = response.data.id
    username=response.data.username
    all_followers[username] = []

    #per ogni follower di kevin scarico a sua volta 1000 suoi followers salvando per ciascuno di essi le stesse info del punto 1:
    for current_user_followers in tweepy.Paginator(client.get_users_followers, id= user_id, user_fields=["description", "protected", "public_metrics"], max_results = 1000): #uso paginator perchè non so a priori quanti follower abbiano
 

        #Dato che user non è serializzabile direttamente come JSON, serve un parsing:
        followers_of_followers=[]
        for follower in current_user_followers.data:
            found_follower = {
                          "id": follower["id"],
                          "name": follower["name"],
                          "username": follower["username"],
                          "description":follower["description"],
                          "public metrics":follower["public metrics"],
                          "protected": follower["protected"],
                         # "last_week_tweets_count": follower["last_week_tweets_count"],  #? ci va anche qui?
                         # "followers": follower["all_followers[username]"]  #?ci va anche qui?
                        }

            
            followers_of_followers.append(found_follower) #inserisco i dati di found_follower nell'array followers_of_followers
       
        all_followers[username] += followers_of_followers #aggiungo l'informazione: username di tutti i followers all'array followers_of_followers
        followers_of_followers[last_week_tweets_count] += last_week_tweets_count #aggiungo l'informazione: last_week_tweets_count all'array followers_of_followers

serialize_json(data_folder, "follower filtrati.json", all_followers)

NotFound: 404 Not Found